# Dataset Preparation

In [2]:
import cv2
import os
import random
import shutil
import pandas as pd
import numpy as np

In [7]:
dataset = "../Dataset/"

In [6]:
right_images = os.listdir(dataset + "Batsman Style Data/right")
left_images = os.listdir(dataset + "Batsman Style Data/left")

In [10]:
os.mkdir(dataset + "Left or Right Data/" + "right")

In [19]:
for image in left_images:
    image_path = dataset + "Batsman Style Data/left/" + image
    
    img = cv2.imread(image_path)
    
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    cv2.imwrite(dataset + "Left or Right Data/" + "left/" + image, grayscale)

In [39]:
count = 0
rand_lst = []
while count<100:
    rand = random.randint(0, len(left_images)-1)
    if rand in rand_lst:
        continue
    rand_lst.append(rand)
    left_img = left_images[rand]
    right_img = right_images[rand]
    
    left_source = dataset + "Left or Right Data/train/left/" + left_img
    right_source = dataset + "Left or Right Data/train/right/" + right_img
    
    left_dest = dataset + "Left or Right Data/val/left/"
    right_dest = dataset + "Left or Right Data/val/right/"
    
    shutil.move(left_source, left_dest)
    shutil.move(right_source, right_dest)
    count += 1

In [13]:
count_res = {}

right_train = os.listdir(dataset + "Left or Right Data/train/right")
left_train = os.listdir(dataset + "Left or Right Data/train/left")

In [6]:
train_images = right_train + left_train

In [21]:
for img in left_train:
    cv_im = cv2.imread(dataset + "Left or Right Data/train/left/" + img)
    shape = str(cv_im.shape)
    
    try:
        count_res[shape] = count_res[shape] + 1
    except:
        count_res[shape] = 1

In [22]:
count_res

{'(197, 88, 3)': 9,
 '(180, 76, 3)': 9,
 '(175, 86, 3)': 20,
 '(170, 84, 3)': 11,
 '(158, 95, 3)': 7,
 '(147, 80, 3)': 16,
 '(191, 91, 3)': 13,
 '(134, 66, 3)': 6,
 '(168, 82, 3)': 16,
 '(159, 78, 3)': 17,
 '(166, 95, 3)': 20,
 '(161, 93, 3)': 10,
 '(144, 63, 3)': 13,
 '(136, 85, 3)': 17,
 '(150, 95, 3)': 20,
 '(156, 90, 3)': 36,
 '(146, 81, 3)': 24,
 '(164, 89, 3)': 18,
 '(194, 96, 3)': 11,
 '(150, 78, 3)': 9,
 '(164, 88, 3)': 9,
 '(117, 75, 3)': 10,
 '(113, 75, 3)': 12,
 '(131, 74, 3)': 14,
 '(124, 89, 3)': 51,
 '(130, 69, 3)': 23,
 '(123, 69, 3)': 8,
 '(167, 120, 3)': 8}

In [27]:
data_lst = ["train", "test", "val"]

for data in data_lst:
    source_images_l = os.listdir(dataset + "Left or Right Data/Raw/" + data + "/left")
    source_images_r = os.listdir(dataset + "Left or Right Data/Raw/" + data + "/right")
    count = 0
    
    for img in source_images_l:
        print(count)
        count += 1
        cv_im = cv2.imread(dataset + "Left or Right Data/Raw/" + data + "/left/" + img)
        cv_resized = cv2.resize(cv_im, (124, 89))
        cv2.imwrite(dataset + "Left or Right Data/Resized/" + data + "/left/" + img, cv_resized)
    
    for img in source_images_r:
        print(count)
        count += 1
        cv_im = cv2.imread(dataset + "Left or Right Data/Raw/" + data + "/right/" + img)
        cv_resized = cv2.resize(cv_im, (124, 89))
        cv2.imwrite(dataset + "Left or Right Data/Resized/" + data + "/right/" + img, cv_resized)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Model Training

In [35]:
{0: "Left", 1:"Right"}

{0: 'Left', 1: 'Right'}

In [45]:
df_train = pd.DataFrame(columns=["Input", "Output"])
df_test = pd.DataFrame(columns=["Input", "Output"])
df_cv = pd.DataFrame(columns=["Input", "Output"])

In [46]:
data_lst = ["train", "test", "val"]

for data in data_lst:
    source_images_l = os.listdir(dataset + "Left or Right Data/Resized/" + data + "/left")
    source_images_r = os.listdir(dataset + "Left or Right Data/Resized/" + data + "/right")
    
    for img in source_images_l:
        cv_im = cv2.imread(dataset + "Left or Right Data/Resized/" + data + "/left/" + img)
        df_dict = {"Input": [cv_im], "Output": [0]}
        df_data = pd.DataFrame(df_dict)
        
        if data == "train":
            df_train = pd.concat([df_train, df_data], axis=0).reset_index(drop=True)
        if data == "test":
            df_test = pd.concat([df_test, df_data], axis=0).reset_index(drop=True)
        if data == "val":
            df_cv = pd.concat([df_cv, df_data], axis=0).reset_index(drop=True)

        
    for img in source_images_r:
        cv_im = cv2.imread(dataset + "Left or Right Data/Resized/" + data + "/right/" + img)
        df_dict = {"Input": [cv_im], "Output": [1]}
        df_data = pd.DataFrame(df_dict)

        
        if data == "train":
            df_train = pd.concat([df_train, df_data], axis=0).reset_index(drop=True)
        if data == "test":
            df_test = pd.concat([df_test, df_data], axis=0).reset_index(drop=True)
        if data == "val":
            df_cv = pd.concat([df_cv, df_data], axis=0).reset_index(drop=True)
      

In [47]:
train_lst = list(df_train["Input"].values)

In [50]:
train_lst[200].shape

(89, 124, 3)

In [1]:
import keras
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from keras.datasets import mnist


11501568/11490434 [==============================] - 2s 0us/step


In [119]:
type(X_train[0])

numpy.ndarray

In [203]:
model = Sequential([
    Conv2D(20, kernel_size=3, activation='relu', input_shape=(89, 124, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(10, kernel_size=3, activation='relu'),
    Conv2D(5, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(2, activation='softmax')
])

In [204]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [156]:
X_train = df_train["Input"].values
y_train = df_train["Output"].values
X_cv = df_cv["Input"].values
y_cv = df_cv["Output"].values
X_test = df_test["Input"].values
y_test = df_test["Output"].values

In [157]:
X_train_np = []
X_cv_np = []
X_test_np = []

In [158]:
for i in range(len(X_train)):
    X_train_np.append(X_train[i])

In [159]:
for i in range(len(X_cv)):
    X_cv_np.append(X_cv[i])

In [160]:
for i in range(len(X_test)):
    X_test_np.append(X_test[i])

In [161]:
X_train = np.array(X_train_np)
X_train.shape

(437, 89, 124, 3)

In [162]:
X_cv = np.array(X_cv_np)
X_cv.shape

(160, 89, 124, 3)

In [163]:
X_test = np.array(X_test_np)
X_test.shape

(40, 89, 124, 3)

In [166]:
X_train = tf.convert_to_tensor(X_train)
X_cv = tf.convert_to_tensor(X_cv)
X_test = tf.convert_to_tensor(X_test)

In [176]:
y_train = y_train.astype("float32")
y_cv = y_cv.astype("float32")
y_test = y_test.astype("float32")

In [199]:
(X_train_m, y_train_m), (X_test_m, y_test_m) = mnist.load_data()


In [205]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=100, batch_size=20)

Epoch 1/100
22/22 [==============================] - 2s 96ms/step - loss: 3.5452 - accuracy: 0.7277 - val_loss: 0.2128 - val_accuracy: 0.9438
Epoch 2/100
22/22 [==============================] - 2s 87ms/step - loss: 0.1557 - accuracy: 0.9451 - val_loss: 0.1165 - val_accuracy: 0.9625
Epoch 3/100
22/22 [==============================] - 2s 82ms/step - loss: 0.0541 - accuracy: 0.9794 - val_loss: 0.1425 - val_accuracy: 0.9688
Epoch 4/100
22/22 [==============================] - 2s 82ms/step - loss: 0.0278 - accuracy: 0.9908 - val_loss: 0.0624 - val_accuracy: 0.9750
Epoch 5/100
22/22 [==============================] - 2s 82ms/step - loss: 0.0078 - accuracy: 0.9977 - val_loss: 0.0528 - val_accuracy: 0.9937
Epoch 6/100
22/22 [==============================] - 2s 88ms/step - loss: 0.0367 - accuracy: 0.9954 - val_loss: 0.0494 - val_accuracy: 0.9937
Epoch 7/100
22/22 [==============================] - 2s 83ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0505 - val_accuracy: 0.9937
Epoch 

22/22 [==============================] - 2s 85ms/step - loss: 6.7369e-06 - accuracy: 1.0000 - val_loss: 0.0983 - val_accuracy: 0.9812
Epoch 58/100
22/22 [==============================] - 2s 85ms/step - loss: 6.5964e-06 - accuracy: 1.0000 - val_loss: 0.0981 - val_accuracy: 0.9812
Epoch 59/100
22/22 [==============================] - 2s 86ms/step - loss: 6.5378e-06 - accuracy: 1.0000 - val_loss: 0.0990 - val_accuracy: 0.9812
Epoch 60/100
22/22 [==============================] - 2s 86ms/step - loss: 6.2565e-06 - accuracy: 1.0000 - val_loss: 0.0997 - val_accuracy: 0.9812
Epoch 61/100
22/22 [==============================] - 2s 86ms/step - loss: 6.1409e-06 - accuracy: 1.0000 - val_loss: 0.1002 - val_accuracy: 0.9812
Epoch 62/100
22/22 [==============================] - 2s 86ms/step - loss: 6.2923e-06 - accuracy: 1.0000 - val_loss: 0.0959 - val_accuracy: 0.9875
Epoch 63/100
22/22 [==============================] - 2s 88ms/step - loss: 6.2524e-06 - accuracy: 1.0000 - val_loss: 0.1002 - val_a

# Inference

In [ ]:
"C:\Users\Assus\Cricket NLP\Cricket NLP Version 1.1\Dataset\Left or Right Data\Resized\test\left\AghhkPjJpVQ_842_845_40.jpg"

In [244]:
img_path = dataset + "Left or Right Data/Resized/test/left/AghhkPjJpVQ_842_845_40.jpg"

In [245]:
img = cv2.imread(img_path)

In [246]:
img.shape

(89, 124, 3)

In [223]:
img = img.resize(img, (124, 89))

TypeError: only integer scalar arrays can be converted to a scalar index

In [254]:
data = {"Input": [img]}
df = pd.DataFrame(data)

In [248]:
test = df["Input"].values

In [249]:
test_list = [test[0]]

In [250]:
test = np.array(test_list)

In [251]:
img = tf.convert_to_tensor(np.array(test))

In [252]:
img.shape

TensorShape([1, 89, 124, 3])

In [253]:
model.predict([img])

array([[1.0000000e+00, 1.3814288e-11]], dtype=float32)

In [255]:
from keras.models import save_model


In [256]:
model.save("Left or right")

INFO:tensorflow:Assets written to: Left or right\assets


In [3]:
from keras.models import load_model


In [4]:
model = load_model("Left or Right")

In [3]:
{0: "Left", 1:"Right"}

In [12]:
def predict_img(path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path, img)
    img = cv2.imread(path)
    try:
        img = cv2.resize(img, (124, 89))
    except Exception as e:
        print(e)
        img = img
        return
    
    data = {"Input": [img]}
    df = pd.DataFrame(data)
    test = df["Input"].values
    test_list = [test[0]]
    test = np.array(test_list)
    img = tf.convert_to_tensor(np.array(test))
    predictions = model.predict([img])
    left = predictions[0][0]
    right = predictions[0][1]
    
    print("Left prob: " + str((left)))
    print("\n")
    print("Right prob: " + str((right)))

In [ ]:
"C:\Users\Assus\Cricket NLP\Cricket NLP Version 1.1\Dataset\Left or Right Data\Screenshot (128).png"

In [14]:
img_path = dataset + "Left or Right Data/Screenshot (128).png"
predict_img(img_path)

Left prob: 5.324265e-06


Right prob: 0.99999464


In [ ]:
"C:\Users\Assus\Cricket NLP\Cricket NLP Version 1.1\Dataset\Left or Right Data\Raw\test_unseen\right\s8OJgjKlxnk_72_74_31.jpg"